<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork899-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Peer Review Assignment - Data Engineer - ETL


Estimated time needed: **20** minutes


## Objectives

In this final part you will:

- Run the ETL process
- Extract bank and market cap data from the JSON file `bank_market_cap.json`
- Transform the market cap currency using the exchange rate data
- Load the transformed data into a seperate CSV


For this lab, we are going to be using Python and several Python libraries. Some of these libraries might be installed in your lab environment or in SN Labs. Others may need to be installed by you. The cells below will install these libraries when executed.


In [ ]:
#!mamba install pandas==1.3.3 -y
#!mamba install requests==2.26.0 -y

## Imports

Import any additional libraries you may need here.


In [164]:
import glob
import pandas as pd
from datetime import datetime
import requests
import wget
import os

As the exchange rate fluctuates, we will download the same dataset to make marking simpler. This will be in the same format as the dataset you used in the last section  


In [14]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv

zsh:1: command not found: wget


zsh:1: command not found: wget
zsh:1: command not found: wget


In [21]:
#data file was downloaded seperately using the code below since wget was not working

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json"

# Make an HTTP GET request to fetch the JSON data
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Get the filename from the URL
    filename = url.split("/")[-1]

    # Save the JSON content to a file
    with open(filename, "wb") as file:
        file.write(response.content)
    print(f"JSON data downloaded and saved as '{filename}'")
else:
    print("Failed to download JSON data.")



## Extract


### JSON Extract Function

This function will extract JSON files.


In [165]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

## Extract Function

Define the extract function that finds JSON file `bank_market_cap_1.json` and calls the function created above to extract data from them. Store the data in a `pandas` dataframe. Use the following list for the columns.


In [166]:
columns=['Name','Market Cap (US$ Billion)']

In [167]:
# first of all, create an emmpty dataframe that will contain the dataframe
# then use glob module to append the data to the emtpy dataframe 
data1 = '/Users/francis/Downloads/source/bank_market_cap_1.json'

def extract():
    # Write your code here
    extracted_data = pd.DataFrame(columns)   #creates an empty dataframe

    for data in glob.glob(data1):            #looks for variable data1 in local directory and append it to the emtpy dataframe
        extracted_data = extracted_data.append(extract_from_json(data), ignore_index=True)
    
    return extracted_data    

<b>Question 1</b> Load the file <code>exchange_rates.csv</code> as a dataframe and find the exchange rate for British pounds with the symbol <code>GBP</code>, store it in the variable  <code>exchange_rate</code>, you will be asked for the number. Hint: set the parameter  <code>index_col</code> to 0.


In [168]:
# Write your code here
data2 = '/Users/francis/Downloads/source/exchange_rates.csv'
df = pd.read_csv(data2, index_col=0)
df

,Rates
AUD,1.297088
BGN,1.608653
BRL,5.409196
CAD,1.271426
CHF,0.886083
CNY,6.483139
CZK,21.510117
DKK,6.119757
EUR,0.822504
GBP,0.732398


In [121]:

exchange_rate = df.loc['GBP', 'Rates']
exchange_rate


0.7323984208000001

## Transform

Using <code>exchange_rate</code> and the `exchange_rates.csv` file find the exchange rate of USD to GBP. Write a transform function that

1. Changes the `Market Cap (US$ Billion)` column from USD to GBP
2. Rounds the Market Cap (US$ Billion)` column to 3 decimal places
3. Rename `Market Cap (US$ Billion)` to `Market Cap (GBP$ Billion)`


In [ ]:
#pd.read_csv(data2)

market_caps = extract()
#market_caps.head()
#market_caps_new= market_caps.drop([0, 1]).reset_index(drop=True)
market_caps_new

In [176]:
def transform(data):
    # Write your code here
    data['Market Cap (US$ Billion)'] *= 0.7768 #Changes the `Market Cap (US$ Billion)` column from USD to GBP with inplace=True
    data['Market Cap (US$ Billion)'] = data['Market Cap (US$ Billion)'].round(decimals=3) #Rounds the Market Cap (US$ Billion)` column to 3 decimal places
    data.rename(columns={'Market Cap (US$ Billion)': 'Market Cap (GBP$ Billion)'}, inplace=True) #NRename `Market Cap (US$ Billion)` to `Market Cap (GBP$ Billion)`


    return data

## Load

Create a function that takes a dataframe and load it to a csv named `bank_market_cap_gbp.csv`. Make sure to set `index` to `False`.


In [177]:
logfile    = "logfile.txt"            # all event logs will be stored in this file
transformed_file = "transformed_data.csv"   # file where transformed data is stored

In [178]:
#load the transformed data (data_to_load) into a new file (transformed files) as shown in the variable above
def load(final_transformed_file, data_to_load):  
    # Write your code here
    data_to_load.to_csv(final_transformed_file)
    
    

## Logging Function


Write the logging function <code>log</code> to log your data:


In [179]:
def log(message):
    # Write your code here
    timestamp_format = "yyyy-MM-dd HH:mm:ss"#get the time format
    now = datetime.now()                     #get the current time
    timestamp = now.strftime(timestamp_format)   #get the timestamp
    with open("logfile.txt", 'a') as f:
        f.write(timestamp + ',' + message + '\n')

## Running the ETL Process


Log the process accordingly using the following <code>"ETL Job Started"</code> and <code>"Extract phase Started"</code>


In [134]:
# Write your code here
log("ETL Job Started")
log("ETL Phase Started")


### Extract


<code>Question 2</code> Use the function <code>extract</code>, and print the first 5 rows, take a screen shot:


In [180]:
# Call the function here
market_caps = extract()
market_caps_new.head()


# Print the rows here
#print(market_caps.head())


,0,Market Cap (US$ Billion),Name
0,NaN,390.934,JPMorgan Chase
1,NaN,345.214,Industrial and Commercial Bank of China
2,NaN,325.331,Bank of America
3,NaN,308.013,Wells Fargo
4,NaN,257.399,China Construction Bank


Log the data as <code>"Extract phase Ended"</code>


In [136]:
# Write your code here
log("Extract phase ended")

### Transform


Log the following  <code>"Transform phase Started"</code>


In [137]:
# Write your code here
log("Transform phases started")

<code>Question 3</code> Use the function <code>transform</code> and print the first 5 rows of the output, take a screen shot:


In [182]:
# Call the function here
result2 = transform(market_caps_new)
result2

# Print the first 5 rows here
first_5_rows = result2.head()
first_5_rows

,0,Market Cap (GBP$ Billion),Name
0,NaN,303.678,JPMorgan Chase
1,NaN,268.162,Industrial and Commercial Bank of China
2,NaN,252.717,Bank of America
3,NaN,239.264,Wells Fargo
4,NaN,199.948,China Construction Bank


Log your data <code>"Transform phase Ended"</code>


In [155]:
# Write your code here
log("Transform phases ended")

### Load


Log the following `"Load phase Started"`.


In [156]:
# Write your code here
log("load phase started")

Call the load function


In [160]:
# Write your code here
loaded_data= load(transformed_file, result2)
print(loaded_data)

None


Log the following `"Load phase Ended"`.


In [ ]:
# Write your code here
log("Load phase ended")

## Authors


Ramesh Sannareddy, Joseph Santrcangelo and Azim Hirjani


### Other Contributors


Rav Ahuja


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                 |
| ----------------- | ------- | ----------------- | ---------------------------------- |
| 2020-11-25        | 0.1     | Ramesh Sannareddy | Created initial version of the lab |


 Copyright © 2020 IBM Corporation. This notebook and its source code are released under the terms of the [MIT License](https://cognitiveclass.ai/mit-license?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork899-2023-01-01&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBM-DA0321EN-SkillsNetwork-21426264&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
